Install stable-baselines3 for training algorithms

In [1]:
!pip install stable_baselines3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 170 kB 8.1 MB/s 
     |████████████████████████████████| 1.5 MB 70.7 MB/s 
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616821 sha256=3a1e415cf60b2a6a065165ec812745cfb4564d336608b53bb69f23aba9cd852a
  Stored in directory: /root/.cache/pip/wheels/27/6d/b3/a3a6e10704795c9b9000f1ab2dc480dfe7bed42f5972806e73
Successfully built gym
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 5.1.0
    Uninstalling importlib-metadata-5.1.0:
      Successfully uninstalled importlib-metadata-5.1.0
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2


Set-up code

In [2]:
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common

!apt-get install -y patchelf

!pip install gym
!pip install free-mujoco-py

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgl1-mesa-dev is already the newest version (20.0.8-0ubuntu1~18.04.1).
libgl1-mesa-dev set to manually installed.
software-properties-common is already the newest version (0.96.24.32.18).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
Suggested packages:
  glew-utils
The following NEW packages will be installed:
  libgl1-mesa-glx libglew-dev libglew2.0 libosmesa6 libosmesa6-dev
0 upgraded, 5 newly installed, 0 to remove and 20 not upgraded.
Need to get 2,916 kB of archives.
After this operation, 12.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libgl1-mesa-glx amd64 20.0.8-0ubuntu1~18.04.1 [5,532 B]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libglew2.0 amd64 2.0.0-5 [140 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/univ

Unzip custom-hopper environment and utils

In [3]:
!unzip custom_hopper.zip

Archive:  custom_hopper.zip
   creating: env/
  inflating: env/__init__.py         
  inflating: env/custom_hopper.py    
  inflating: env/mujoco_env.py       
   creating: env/assets/
  inflating: env/assets/hopper.xml   


Import libraries

In [4]:
import gym
from env.custom_hopper import *
from stable_baselines3 import PPO
from itertools import product
import numpy as np

Compiling /usr/local/lib/python3.8/dist-packages/mujoco_py/cymj.pyx because it changed.
[1/1] Cythonizing /usr/local/lib/python3.8/dist-packages/mujoco_py/cymj.pyx
running build_ext
building 'mujoco_py.cymj' extension
creating /usr/local/lib/python3.8/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_38_linuxcpuextensionbuilder
creating /usr/local/lib/python3.8/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_38_linuxcpuextensionbuilder/temp.linux-x86_64-3.8
creating /usr/local/lib/python3.8/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_38_linuxcpuextensionbuilder/temp.linux-x86_64-3.8/usr
creating /usr/local/lib/python3.8/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_38_linuxcpuextensionbuilder/temp.linux-x86_64-3.8/usr/local
creating /usr/local/lib/python3.8/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_38_linuxcpuextensionbuilder/temp.linux-x86_64-3.8/usr/local/lib
creating /usr/local/lib/python3.8/dist-packages/mujoco_py/generated/_pyxbld_2.0.2.13_38_linuxcpuexten

Hyperparameters intervals

In [7]:
n_steps_ls = np.linspace(32, 5000, num=3, dtype=int)
batch_size_ls = np.linspace(4, 4096, num=3, dtype=int)
n_epochs_ls = np.linspace(3, 30, num=3, dtype=int)
learning_rate_ls = np.linspace(5e-6, 0.003, num=3)
ent_coef_ls = np.linspace(0, 0.01, num=3)

In [6]:
train_env = 'source'
env = gym.make(f'CustomHopper-{train_env}-v0')

cart_prod = product(n_steps_ls, batch_size_ls, n_epochs_ls, learning_rate_ls, ent_coef_ls)
with open('configurations.txt', 'w') as f:
    for i, line in enumerate(cart_prod):
        f.write(f"Config{i}: {line}\n")
    f.close()

config = 1
for n_steps, batch_size, n_epochs, learning_rate, ent_coef in product(n_steps_ls, batch_size_ls, n_epochs_ls, learning_rate_ls, ent_coef_ls):

    model = PPO("MlpPolicy", env=env, verbose=1, n_steps=n_steps, batch_size=batch_size, n_epochs=n_epochs, learning_rate=learning_rate, ent_coef=ent_coef)

    model.learn(total_timesteps=100000)
    model.save(f'trained_models/PPO_config{config}_{train_env}')
    config += 1




Output streaming troncato alle ultime 5000 righe.
|    loss                 | 35.8         |
|    n_updates            | 6879         |
|    policy_gradient_loss | -9.01e-05    |
|    std                  | 0.994        |
|    value_loss           | 150          |
------------------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 60.7          |
|    ep_rew_mean          | 99.3          |
| time/                   |               |
|    fps                  | 187           |
|    iterations           | 2295          |
|    time_elapsed         | 392           |
|    total_timesteps      | 73440         |
| train/                  |               |
|    approx_kl            | 2.4400651e-07 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -4.24         |
|    explained_variance   | 0.00175       |
|    learning_rate        | 5e-0

KeyboardInterrupt: ignored